#### Applied Data Science Capstone: Neighborhoods in Toronto - Part 1 & Part 2

## Part1

###### 1. Import all required libraries

In [172]:
# Import necessary libraries

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library


###### 2. Use beautiful soup to scrape data

In [154]:
# send the GET request
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [155]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [156]:
# create three lists to store table data
postalCodeList = []
boroughList = []
neighborhoodList = []

In [157]:
# append the data into the respective lists
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text.rstrip('\n')) # avoid new lines in neighborhood cell

In [158]:
# create a new DataFrame from the three lists
toronto_df = pd.DataFrame({"PostalCode": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})

toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,
1,M2A\n,Not assigned\n,
2,M3A\n,North York\n,Parkwoods
3,M4A\n,North York\n,Victoria Village
4,M5A\n,Downtown Toronto\n,Regent Park / Harbourfront


##### 3. Drop cell with "Not assigned" value

In [159]:
# drop cells with a borough that is Not assigned
toronto_df_dropna = toronto_df[toronto_df.Borough != "Not assigned"].reset_index(drop=True)
toronto_df_dropna.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,
1,M2A\n,Not assigned\n,
2,M3A\n,North York\n,Parkwoods
3,M4A\n,North York\n,Victoria Village
4,M5A\n,Downtown Toronto\n,Regent Park / Harbourfront


In [160]:
# drop cells with a borough that is Not assigned
toronto_df_dropna = toronto_df[toronto_df.Borough != "Not assigned"].reset_index(drop=True)
toronto_df_dropna.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,
1,M2A\n,Not assigned\n,
2,M3A\n,North York\n,Parkwoods
3,M4A\n,North York\n,Victoria Village
4,M5A\n,Downtown Toronto\n,Regent Park / Harbourfront


In [161]:

toronto_df_grouped = toronto_df_dropna.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,
1,M1B\n,Scarborough\n,Malvern / Rouge
2,M1C\n,Scarborough\n,Rouge Hill / Port Union / Highland Creek
3,M1E\n,Scarborough\n,Guildwood / Morningside / West Hill
4,M1G\n,Scarborough\n,Woburn


In [162]:
toronto_df_grouped.drop(toronto_df_grouped[ toronto_df_grouped['Borough'] == 'Not assigned\n' ].index , inplace=True)

In [163]:
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
1,M1B\n,Scarborough\n,Malvern / Rouge
2,M1C\n,Scarborough\n,Rouge Hill / Port Union / Highland Creek
3,M1E\n,Scarborough\n,Guildwood / Morningside / West Hill
4,M1G\n,Scarborough\n,Woburn
5,M1H\n,Scarborough\n,Cedarbrae


###### 4. strip /n from data

In [164]:
#grouping neighbourhood in sme borough
toronto_df_grouped= toronto_df_grouped.replace('\n',' ', regex=True)
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
1,M1B,Scarborough,Malvern / Rouge
2,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
3,M1E,Scarborough,Guildwood / Morningside / West Hill
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae


In [165]:
# for Neighborhood="Not assigned", make the value the same as Borough
for index, row in toronto_df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
1,M1B,Scarborough,Malvern / Rouge
2,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
3,M1E,Scarborough,Guildwood / Morningside / West Hill
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae


###### 5. Group neighborhood of same borough

In [166]:
# for Neighborhood="Not assigned", make the value the same as Borough
for index, row in toronto_df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
1,M1B,Scarborough,Malvern / Rouge
2,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
3,M1E,Scarborough,Guildwood / Morningside / West Hill
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae


###### 6. Number of rows in dataframe

In [167]:
# print the number of rows of the cleaned dataframe
toronto_df_grouped.shape

(103, 3)

## Part 2

###### 7. Load csv file with coordinates from  Coursera

In [168]:
# load the coordinates from the csv file on Coursera
coordinates = pd.read_csv("Geospatial_Coordinates.csv")
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


###### 8. Merge two csv data

In [173]:
Merged_data = toronto_df_grouped.join(coordinates)
Merged_data

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
1,M1B,Scarborough,Malvern / Rouge,M1C,43.784535,-79.160497
2,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,M1E,43.763573,-79.188711
3,M1E,Scarborough,Guildwood / Morningside / West Hill,M1G,43.770992,-79.216917
4,M1G,Scarborough,Woburn,M1H,43.773136,-79.239476
5,M1H,Scarborough,Cedarbrae,M1J,43.744734,-79.239476
6,M1J,Scarborough,Scarborough Village,M1K,43.727929,-79.262029
7,M1K,Scarborough,Kennedy Park / Ionview / East Birchmount Park,M1L,43.711112,-79.284577
8,M1L,Scarborough,Golden Mile / Clairlea / Oakridge,M1M,43.716316,-79.239476
9,M1M,Scarborough,Cliffside / Cliffcrest / Scarborough Village West,M1N,43.692657,-79.264848
10,M1N,Scarborough,Birch Cliff / Cliffside West,M1P,43.757410,-79.273304


In [174]:
# save data to csv file
Merged_data.to_csv(r'df_cord.csv',index=False)